<a href="https://colab.research.google.com/github/632002nancy/Angular/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import torch
import numpy as np
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [2]:
device = torch.device('cuda')

# Load Pre-trained GPT2 Model

In [3]:
MODEL_NAME = 'gpt2'

model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

### Generating from vanilla model before fine-tuning

In [4]:
input_text = ['(a) Definitions.--In this section: (1) Business entity.--The term ``business entity'' means a firm, corporation, association, partnership, consortium, joint venture, or other form of enterprise. (2) Facility.--The term ``facility'' means any real property, including any building, improvement, or appurtenance. (3) Gross negligence.--The term ``gross negligence'' means voluntary and conscious conduct by a person with knowledge (at the time of the conduct) that the conduct is likely to be harmful to the health or well-being of another person. (4) Intentional misconduct.--The term ``intentional misconduct'' means conduct by a person with knowledge (at the time of the conduct) that the conduct is harmful to the health or well-being of another person. (5) Nonprofit organization.--The term ``nonprofit organization'' means-- (A) any organization described in section 501(c)(3) of the Internal Revenue Code of 1986 and exempt from tax under section 501(a) of such Code; or (B) any not-for-profit organization organized and conducted for public benefit and operated primarily for charitable, civic, educational, religious, welfare, or health purposes. (6) State.--The term ``State'' means each of the several States, the District of Columbia, the Commonwealth of Puerto Rico, the Virgin Islands, Guam, American Samoa, the Northern Mariana Islands, any other territory or possession of the United States, or any political subdivision of any such State, territory, or possession. (b) Limitation on Liability.-- (1) In general.--Subject to subsection (c), a business entity shall not be subject to civil liability relating to any injury or death occurring at a facility of the business entity in connection with a use of such facility by a nonprofit organization if-- (A) the use occurs outside of the scope of business of the business entity; (B) such injury or death occurs during a period that such facility is used by the nonprofit organization; and (C) the business entity authorized the use of such facility by the nonprofit organization. (2) Application.--This subsection shall apply-- (A) with respect to civil liability under Federal and State law; and (B) regardless of whether a nonprofit organization pays for the use of a facility. (c) Exception for Liability.--Subsection (b) shall not apply to an injury or death that results from an act or omission of a business entity that constitutes gross negligence or intentional misconduct, including any misconduct that-- (1) constitutes a crime of violence (as that term is defined in section 16 of title 18, United States Code) or act of international terrorism (as that term is defined in section 2331 of title 18) for which the defendant has been convicted in any court; (2) constitutes a hate crime (as that term is used in the Hate Crime Statistics Act (28 U.S.C. 534 note)); (3) involves a sexual offense, as defined by applicable State law, for which the defendant has been convicted in any court; or (4) involves misconduct for which the defendant has been found to have violated a Federal or State civil rights law. (d) Superseding Provision.-- (1) In general.--Subject to paragraph (2) and subsection (e), this Act preempts the laws of any State to the extent that such laws are inconsistent with this Act, except that this Act shall not preempt any State law that provides additional protection from liability for a business entity for an injury or death with respect to which conditions under subparagraphs (A) through (C) of subsection (b)(1) apply. (2) Limitation.--Nothing in this Act shall be construed to supersede any Federal or State health or safety law. (e) Election of State Regarding Nonapplicability.--This Act shall not apply to any civil action in a State court against a business entity in which all parties are citizens of the State if such State enacts a statute-- (1) citing the authority of this subsection; (2) declaring the election of such State that this Act shall not apply to such civil action in the State; and (3) containing no other provision.']

for t in input_text:

    inputs = tokenizer(f'{t}',
                       return_tensors='pt').to(device)

    generated = model.generate(**inputs,
                               top_p=0.7,
                               temperature=0.5,
                               do_sample=True,
                               repetition_penalty=4.0,
                               max_length = 1000)

    generated = tokenizer.batch_decode(generated)[0]
    print(generated)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


(a) Definitions.--In this section: (1) Business entity.--The term ``business entity means a firm, corporation, association, partnership, consortium, joint venture, or other form of enterprise. (2) Facility.--The term ``facility means any real property, including any building, improvement, or appurtenance. (3) Gross negligence.--The term ``gross negligence means voluntary and conscious conduct by a person with knowledge (at the time of the conduct) that the conduct is likely to be harmful to the health or well-being of another person. (4) Intentional misconduct.--The term ``intentional misconduct means conduct by a person with knowledge (at the time of the conduct) that the conduct is harmful to the health or well-being of another person. (5) Nonprofit organization.--The term ``nonprofit organization means-- (A) any organization described in section 501(c)(3) of the Internal Revenue Code of 1986 and exempt from tax under section 501(a) of such Code; or (B) any not-for-profit organizatio

# Finetuning GPT2 to summarise the given text.

In [5]:
speech_data = pd.read_parquet("0000.parquet")[:500]
speech_data.shape

(500, 3)

In [9]:
speech_data

,text,summary,title
0,SECTION 1. LIABILITY OF BUSINESS ENTITIES PROV...,Shields a business entity from civil liability...,A bill to limit the civil liability of busines...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Human Rights Information Act - Requires certai...,Human Rights Information Act
2,SECTION 1. SHORT TITLE.\n\n This Act may be...,Jackie Robinson Commemorative Coin Act - Direc...,Jackie Robinson Commemorative Coin Act
3,SECTION 1. NONRECOGNITION OF GAIN WHERE ROLLOV...,Amends the Internal Revenue Code to provide (t...,To amend the Internal Revenue Code to provide ...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Native American Energy Act - (Sec. 3) Amends t...,Native American Energy Act
...,...,...,...
495,SECTION 1. SHORT TITLE.\n This Act may be c...,(This measure has not been amended since it wa...,Department of Homeland Security Headquarters C...
496,SECTION 1. SHORT TITLE.\n\n This Act may be...,Red River Private Property Protection Act - (S...,Red River Private Property Protection Act
497,SECTION 1. SHORT TITLE.\n\n This Act may be...,Cheaper Car Insurance Act of 2010 - Amends the...,A bill to create a penalty for automobile insu...
498,SECTION 1. SHORT TITLE.\n\n This Act may be...,Post-Abortion Depression Research and Care Act...,"To provide for research on, and services for i..."


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
ls

0000.parquet  data/  data.txt  drive/  fine_tune.ipynb  run_clm.py  sample_data/  speech_logs/


# Preparing the data and tokenizer

Adding special tokens to highlight question and answer

In [10]:
data_txt = []
for i in range(speech_data.shape[0]):
    o, p = speech_data.text[i], speech_data.summary[i]
    line = f'@@Q_START@@ {o} @@Q_END@@ @@A_START@@ {p} @@A_END@@'
    data_txt.append(line)

In [11]:
data_txt[0]

"@@Q_START@@ SECTION 1. LIABILITY OF BUSINESS ENTITIES PROVIDING USE OF FACILITIES \n              TO NONPROFIT ORGANIZATIONS.\n\n    (a) Definitions.--In this section:\n            (1) Business entity.--The term ``business entity'' means a \n        firm, corporation, association, partnership, consortium, joint \n        venture, or other form of enterprise.\n            (2) Facility.--The term ``facility'' means any real \n        property, including any building, improvement, or appurtenance.\n            (3) Gross negligence.--The term ``gross negligence'' means \n        voluntary and conscious conduct by a person with knowledge (at \n        the time of the conduct) that the conduct is likely to be \n        harmful to the health or well-being of another person.\n            (4) Intentional misconduct.--The term ``intentional \n        misconduct'' means conduct by a person with knowledge (at the \n        time of the conduct) that the conduct is harmful to the health \n        o

In [12]:
with open('data.txt', 'w') as w:
    for ex in data_txt:
        w.write(f'{ex}\n')

In [13]:
! mkdir -p data/new_tokenizer

In [14]:
# Adding the new tokens and saving the new tokenizer

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_tokens(['@@Q_START@@', '@@Q_END@@', '@@A_START@@', '@@A_END@@'], special_tokens=True)


tokenizer.save_pretrained('data/new_tokenizer')

('data/new_tokenizer/tokenizer_config.json',
 'data/new_tokenizer/special_tokens_map.json',
 'data/new_tokenizer/vocab.json',
 'data/new_tokenizer/merges.txt',
 'data/new_tokenizer/added_tokens.json')

# Training script

Using gradient accumulation to fit into 12 gb GPU memory.

In [15]:
!pip install datasets 'transformers[torch]' evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 32.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.

In [17]:
!python run_clm.py \
   --do_train \
   --model_name_or_path "gpt2" \
   --tokenizer_name "data/new_tokenizer" \
   --model_type "gpt2" \
   --train_file "data.txt" \
   --validation_file "data.txt" \
   --cache_dir "data/cache" \
   --output_dir "speech_logs" \
   --per_device_train_batch_size 2 \
   --gradient_accumulation_steps 8 \
   --save_total_limit 3 \
   --save_strategy "steps" \
   --eval_steps 500 \
   --save_steps 500 \
   --preprocessing_num_workers 4 \
   --num_train_epochs 3 \
   --block_size 512

2024-05-27 05:44:11.627452: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 05:44:11.628079: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 05:44:11.630233: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-27 05:44:11.642359: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 05:44:12.888236: W tensorflow/comp

# Load Trained Model

In [19]:
checkpoint_dir = 'speech_logs/checkpoint-500'

model = GPT2LMHeadModel.from_pretrained(checkpoint_dir).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
# Memorize special tokens IDs
vocab = tokenizer.get_vocab()
id0, id1, id2, id3 = vocab['@@Q_START@@'], vocab['@@Q_END@@'], vocab['@@A_START@@'], vocab['@@A_END@@']
id0, id1, id2, id3

(50257, 50258, 50259, 50260)

### Generation after fine-tune

In [21]:
input_text = ['(a) Definitions.--In this section: (1) Business entity.--The term ``business entity'' means a firm, corporation, association, partnership, consortium, joint venture, or other form of enterprise. (2) Facility.--The term ``facility'' means any real property, including any building, improvement, or appurtenance. (3) Gross negligence.--The term ``gross negligence'' means voluntary and conscious conduct by a person with knowledge (at the time of the conduct) that the conduct is likely to be harmful to the health or well-being of another person. (4) Intentional misconduct.--The term ``intentional misconduct'' means conduct by a person with knowledge (at the time of the conduct) that the conduct is harmful to the health or well-being of another person. (5) Nonprofit organization.--The term ``nonprofit organization'' means-- (A) any organization described in section 501(c)(3) of the Internal Revenue Code of 1986 and exempt from tax under section 501(a) of such Code; or (B) any not-for-profit organization organized and conducted for public benefit and operated primarily for charitable, civic, educational, religious, welfare, or health purposes. (6) State.--The term ``State'' means each of the several States, the District of Columbia, the Commonwealth of Puerto Rico, the Virgin Islands, Guam, American Samoa, the Northern Mariana Islands, any other territory or possession of the United States, or any political subdivision of any such State, territory, or possession. (b) Limitation on Liability.-- (1) In general.--Subject to subsection (c), a business entity shall not be subject to civil liability relating to any injury or death occurring at a facility of the business entity in connection with a use of such facility by a nonprofit organization if-- (A) the use occurs outside of the scope of business of the business entity; (B) such injury or death occurs during a period that such facility is used by the nonprofit organization; and (C) the business entity authorized the use of such facility by the nonprofit organization. (2) Application.--This subsection shall apply-- (A) with respect to civil liability under Federal and State law; and (B) regardless of whether a nonprofit organization pays for the use of a facility. (c) Exception for Liability.--Subsection (b) shall not apply to an injury or death that results from an act or omission of a business entity that constitutes gross negligence or intentional misconduct, including any misconduct that-- (1) constitutes a crime of violence (as that term is defined in section 16 of title 18, United States Code) or act of international terrorism (as that term is defined in section 2331 of title 18) for which the defendant has been convicted in any court; (2) constitutes a hate crime (as that term is used in the Hate Crime Statistics Act (28 U.S.C. 534 note)); (3) involves a sexual offense, as defined by applicable State law, for which the defendant has been convicted in any court; or (4) involves misconduct for which the defendant has been found to have violated a Federal or State civil rights law. (d) Superseding Provision.-- (1) In general.--Subject to paragraph (2) and subsection (e), this Act preempts the laws of any State to the extent that such laws are inconsistent with this Act, except that this Act shall not preempt any State law that provides additional protection from liability for a business entity for an injury or death with respect to which conditions under subparagraphs (A) through (C) of subsection (b)(1) apply. (2) Limitation.--Nothing in this Act shall be construed to supersede any Federal or State health or safety law. (e) Election of State Regarding Nonapplicability.--This Act shall not apply to any civil action in a State court against a business entity in which all parties are citizens of the State if such State enacts a statute-- (1) citing the authority of this subsection; (2) declaring the election of such State that this Act shall not apply to such civil action in the State; and (3) containing no other provision.']

for t in input_text:

    inputs = tokenizer(f'@@Q_START@@ {t} @@Q_END@@ @@A_START@@',
                       return_tensors='pt').to(device)

    generated = model.generate(**inputs,
                               top_p=0.7,
                               temperature=0.5,
                               do_sample=True,
                               eos_token_id=id3,
                               repetition_penalty=4.0,
                               max_length = 1000)

    generated = tokenizer.batch_decode(generated)[0]
    answer = re.search('@@A_START@@(.*)', generated).group(1)
    answer = re.sub('@@A_END@@', '', answer)

    print(f'QUESTION: {t}\nANSWER: {answer}', end='\n')

Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


QUESTION: (a) Definitions.--In this section: (1) Business entity.--The term ``business entity means a firm, corporation, association, partnership, consortium, joint venture, or other form of enterprise. (2) Facility.--The term ``facility means any real property, including any building, improvement, or appurtenance. (3) Gross negligence.--The term ``gross negligence means voluntary and conscious conduct by a person with knowledge (at the time of the conduct) that the conduct is likely to be harmful to the health or well-being of another person. (4) Intentional misconduct.--The term ``intentional misconduct means conduct by a person with knowledge (at the time of the conduct) that the conduct is harmful to the health or well-being of another person. (5) Nonprofit organization.--The term ``nonprofit organization means-- (A) any organization described in section 501(c)(3) of the Internal Revenue Code of 1986 and exempt from tax under section 501(a) of such Code; or (B) any not-for-profit o